In [24]:
import sys
import os
import cv2
import numpy as np
import random
import math
# from matplotlib import pyplot as plt
from IPython.display import clear_output

# %matplotlib inline

### 5.2 Intensity based Object Tracking

In [28]:
# Setup for object tracking

if not os.path.isdir(os.path.join(os.getcwd(), 'frames')):
    os.mkdir("frames")
else:
    print('frames already exists')

if not os.path.isdir(os.path.join(os.getcwd(), 'composite')):
    os.mkdir("composite")
else:
    print('composite already exists')
    
framenumber = 0
# framenumber = 66
framectr = 0
# omovie = cv2.VideoCapture('combination.mov')
omovie = cv2.VideoCapture('Opt1-MarionetteMovements.mov')
frame_height = omovie.get(cv2.CAP_PROP_FRAME_HEIGHT)
frame_width = omovie.get(cv2.CAP_PROP_FRAME_WIDTH)

# Extract the frames from original video
while(1):
# while(framenumber<50):
    ret, frame = omovie.read()
    if not ret:
        break
    print('Extracting: %d' % framenumber)
    clear_output(wait=True)
    cv2.imwrite('frames/%d.tif' % framenumber, frame)
    framenumber += 1
omovie.release()
print("done")


done


# A2

WAIT FOR THE FRAMES FOLDER TO GENERATEFIRST!!

In [27]:
def fits(frame, y_c, x_c, k):
    #BGR
    lower_y = y_c-k
    upper_y = y_c+k
    lower_x = x_c-k
    upper_x = x_c+k
    if(lower_y<0):
        lower_y = 0
    if(upper_y>frame.shape[0]):
        upper_y = frame.shape[0]
    if(lower_x<0):
        lower_x = 0
    if(upper_x>frame.shape[1]):
        upper_x = frame.shape[1]
    num = 0 
    for y in range(lower_y,upper_y):
        for x in range(lower_x, upper_x):
            if frame[y][x][2]>100 and frame[y][x][0]<frame[y][x][2]-50 and frame[y][x][1]<frame[y][x][2]-50:
                num+=1
#             if frame[y][x][0]<80:
#                 if frame[y][x][1]<80:
#                     if frame[y][x][2]>120:
#                         num+=1
    return num
                
                
    

In [17]:
process_frame =0 # 

k=3

# Set up the source with the first frame 
start = 'frames/'+str(process_frame)+'.tif'
first_frame = cv2.imread(start)
print(first_frame.shape) # width is 1.25 times the height
height = first_frame.shape[0] 
width = first_frame.shape[1] 

red = []
framectr = framenumber-1
# framectr=1
while process_frame <=framectr:
    red.append([])
    current_frame = cv2.imread('frames/'+str(process_frame)+'.tif')
    new_frame = cv2.imread('frames/'+str(process_frame)+'.tif')
    for y in range(0,height,k):
        for x in range(0,width,k): 
#             print('Fram: %d Rows in frame: %d, progress: %.2f %%' % (process_frame, y*width+x, 100*(y*width+x)/(height*width)))
#             clear_output(wait=True)
            redness = fits(current_frame, y, x, k)
            if(redness>30):
#                 print("foundit")
                red[process_frame].append([y,x])
                new_frame[y][x]= [0,0,0]
    cv2.imwrite('composite/'+str(process_frame)+'.tif', new_frame)
    process_frame+=1
    print("end of a frame"+str(process_frame))
    clear_output(wait=True)


    
    
    
print("Resetting demofile")
f = open("demofile.txt", "w")
f.write("Test!\n")
f.close()

f = f = open("demofile.txt", "a")
# f.write("test\n")
# f.write("test\n")
counter=0
for fr in red:
    f.write("New Frame"+str(counter)+"\n")
    counter+=1
    for point in fr:
        f.write(str(point)+" ")
    print("\n")
f.close()



    


Resetting demofile























































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































In [18]:
print("starting grouping")
#Determine the average points for red. 

groupedRed = []
for fr in range(len(red)): # Go through each frame
    print("FRAME "+str(fr))
    clear_output(wait=True)
    maxgroup = 0
    frame = red[fr].copy()
    groups = []
    groups.append([  [frame[0],0] ])
    q = []
    q.append([frame[0],0])
    frame.pop(0)
    while len(q)>0:
        curr=q[0][0]
        curr_group = q[0][1]
        q.pop(0)
        #search through frame, and find any connected points
        t = frame.copy()
        for i in t:
#             print(i)
#             print(curr)
#             print(i[0]>=(curr[0]-2*k))
#             print( i[1]<=(curr[1]+2*k))
            #Find values within 2 spaces in both axes of it
            if i[0]>=(curr[0]-2*k) and i[0]<=(curr[0]+2*k) and i[1]>=(curr[1]-2*k) and i[1]<=(curr[1]+2*k):
                groups[curr_group].append([[i[0],i[1]], curr_group])
                q.append( [[i[0],i[1]], curr_group] )
                frame.remove(i)
        #If empty, add start of group in
        if len(q)==0:
            if len(frame)==0:
                pass
            else:
                maxgroup+=1
                q.append([frame[0],maxgroup])
                groups.append([    [frame[0],maxgroup]   ])
                frame.pop(0)
        
    
    groupedRed.append(groups)
ave = []

for fr in groupedRed:    
    averages = []
    for g in fr :
        y = 0
        x = 0
        for point in g : 
            y+= point[0][0]
            x+= point[0][1]
        averages.append([y/len(g), x/len(g), len(g)])
    ave.append(averages)
print("AVERTAGE FIUYD++")

f = open("averages.txt","w")
f.write("average position of stickers on frame")
f.close()
f = open("averages.txt","a")
counter=0
for fr in ave:
    f.write("Frame "+str(counter)+"\n")
    f.write(str(fr))
    f.write("\n")
    counter+=1
f.close()
    
f = open("groups.txt", "w")
f.write(str(groupedRed))
f.close()

AVERTAGE FIUYD++


In [19]:
# Take the data from the file, and work out the 5 best, then slot them into torso, l, r, fl, fr
f = open("averages.txt","r")
lines = f.readlines()
frames = (len(lines))/2
# print(len(lines))
# print(frames)
# print(lines[-1])
# print(lines[-2])
# print(lines[-2].split())
# print(lines[-2].split()[-1])
print("starting")

f2 = open("ordered.txt","w")
f2.write("")
f2.close()
f2 = open("ordered.txt","a")

f3 = open("top5.txt","w")
f3.write("")
f3.close()
f3 = open("top5.txt","a")
for l in range(1,len(lines),2):
    f2.write("Frame"+str((l-1)/2)+"\n")
    f3.write("Frame"+str(int((l-1)/2)))
#     print(lines[l])
#     This generates the groups for each frame 
    grps = lines[l][1:-3] # ]]\n is being ignored from the end 
    grps=grps.replace("[",'')
    grps = grps.split("], ")
#     print(grps)
#     print("\n")
    most = []
    for gr in grps:
#         print("points and most")
        pts = gr.split(',')
        pts[0]=float(pts[0])
        pts[1]=float(pts[1])
        pts[2]=float(pts[2])
        
#         print(pts, most)
#         print(float(pts[0]))
#         print(float(pts[1]))
#         print(float(pts[2]))
        length = float(pts[2])
        if most==[]:
            most.append(pts)
        else:
            inthere = False 
            for i in range(0,len(most)):
                if length> float(most[i][2]) and inthere==False:
                    most.insert(i,pts)
                    inthere = True
            if inthere==False:
                most.append(pts)
    #Now, most is a list ordered by number of points in cluster.
#     print("most")
#     print(most)
    counter = 0
    
    
    for i in most:
        f2.write("_")
        f2.write(str(i))
        if counter<5 and i[2]>8:
#             print(i)
            f3.write("_")
            f3.write(str(i))
            counter+=1
            
    f2.write("\n")
    f3.write("\n")
    
    
    
    

f3.close()
f2.close()    
f.close()

starting


In [20]:
# Asign first frame to lf, rh,lh,rf,t 
# These are left foot, right, left, right foot, torso

body = [] # t, hands, feet

#body[i] is frame i 
#body[i][0] = [torso]
f = open("top5.txt","r")



lines = f.readlines()
f.close()
for l in range(0,len(lines)):
# for l in range(0,9):
    body.append([])
    frame = lines[l][0:-1]
    pts = frame.split("_")
    pts = pts[1:len(pts)]
#     if len(pts)<=3:
#         print(str(l)+" "+str(len(pts)))
#     print(str(l)+" "+str(len(pts)))
    ylist = []
    for p in range(0,len(pts)):
        pts[p]=pts[p][1:-1].split(',')
        pts[p][0] = float(pts[p][0])
        pts[p][1] = float(pts[p][1])
        pts[p][2] = float(pts[p][2])
        print("pts[p]")
        print(pts[p])
        
        if ylist==[]:
            ylist.append(pts[p])
        else:
            inthere = False
            for i in range(0,len(ylist)):
                if pts[p][0]<ylist[i][0] and inthere==False:
                    ylist.insert(i,pts[p])
                    inthere = True
            if inthere==False:
                ylist.append(pts[p])
#     print("ylist")
#     print(ylist)
#     if l==36:
#         print(len(ylist))

    if body==[[]]:
        body[l].append(ylist[2]) 
        body[l].append(ylist[1])
        body[l].append(ylist[0])
        body[l].append(ylist[4])
        body[l].append(ylist[3])
        continue
    # Lefts to the left and Rights to the right
    # Hands and torso above the feet
    # The lower distance is a priority
    if len(ylist)==5:
#         body[l]=body[l-1]
        ltr=[]
        for i in range(0,3): # Upper
            if ltr==[]:
                ltr.append(ylist[i])
            else:
                inthere=False
                for j in range(0,len(ltr)):
                    if ylist[i][1]<ltr[j][1] and inthere==False:
                        ltr.insert(j,ylist[i])
                        inthere=True
                if inthere==False:
                    ltr.append(ylist[i])
#         print("ltr")
#         print(ltr)
        body[l].append(ltr[1])
        body[l].append(ltr[0])
        body[l].append(ltr[2])
        body[l].append(ylist[3])
        body[l].append(ylist[4]) 
    if len(ylist)<=4 and len(ylist)>=2:
        # Compare all the points with every part
        # If torso missing, just shift it from previous point towards average of new points OR just leave it be
        #torso
#         print(body[l-1])
#         if(l==5 or l==7):
#             print("ylist is still")
#             print(ylist)
        
        torso = []
        #hands
        hand1 = []
        hand2 = []
        #feet
        foot1 = []
        foot2 = []
        listed = [torso,hand1,hand2,foot1,foot2]
        temp = [[],[],[],[],[]]
        for i in ylist:
#             if(l==5 or l==7):
#                 print("l is "+str(l)+", and i is:")
#                 print(i)
            torso.append(math.sqrt( (body[l-1][0][0]-i[0])**2+(body[l-1][0][1]-i[1])**2  ))
            hand1.append(math.sqrt( (body[l-1][1][0]-i[0])**2+(body[l-1][1][1]-i[1])**2  ))
            hand2.append(math.sqrt( (body[l-1][2][0]-i[0])**2+(body[l-1][2][1]-i[1])**2  ))
            foot1.append(math.sqrt( (body[l-1][3][0]-i[0])**2+(body[l-1][3][1]-i[1])**2  ))
            foot2.append(math.sqrt( (body[l-1][4][0]-i[0])**2+(body[l-1][4][1]-i[1])**2  ))
            
            mini = min(torso[-1], hand1[-1], hand2[-1], foot1[-1], foot2[-1])
            if mini == torso[-1]:
                print('torso')
                temp[0] = i
                for j in listed:
                    if j==torso:
                        listed.remove(torso)
                        break
            if mini == hand1[-1]:
                print('hand1')
                temp[1] = i
                for j in listed:
                    if j==hand1:
                        listed.remove(hand1)
                        break
            if mini == hand2[-1]:
                print('hand2')
                temp[2] = i
                for j in listed:
                    if j==hand2:
                        listed.remove(hand2)
                        break
            if mini == foot1[-1]:
                print('foot1')
                temp[3] = i
                for j in listed:
                    if j==foot1:
                        listed.remove(foot1)
                        break
            if mini == foot2[-1]:
                print('foot2')
                temp[4] = i
                for j in listed:
                    if j==foot2:
                        listed.remove(foot2)
                        break
        print("converting")
        
        print(temp)
        if temp[0]==[]:
            temp[0]=body[l-1][0]
        if temp[1]==[] and temp[2]==[]:
            temp[1]=body[l-1][1]
            temp[2]=body[l-1][2]
        if temp[3]==[] and temp[4]==[]:
            temp[3]=body[l-1][3]
            temp[4]=body[l-1][4]
        if temp[1]==[]:
#             print("converting hand1")
            temp[1]=temp[2]
        if temp[2]==[]:
#             print("converting hand2")
            temp[2]=temp[1]
        if temp[3]==[]:
#             print('conmverting foot1')
            temp[3]=temp[4]
        if temp[4]==[]:
#             print('converting foot2')
#             print(temp[3])
            temp[4]=temp[3]
        body[l]=temp
#         print("temp\n")
#         print(temp)
    if len(ylist)==1:
        print("111111\n\n")
        body[l]=body[l-1]
        #Shift torso, repeat for others. 
        pass
    print(body[l])
    print('\n')
    

f = open("mario/data/positions.txt","w")
f.write("")
f.close()

f = open("mario/data/positions.txt","a")
counter = 0
for frame in body:
    f.write("Frame"+str(counter))
    for i in frame:
        f.write('_')
        f.write(str(i))   
    f.write('\n')
    counter+=1
f.close()  
    
            
            

pts[p]
[261.44262295081967, 230.01639344262296, 61.0]
pts[p]
[80.01923076923077, 280.38461538461536, 52.0]
pts[p]
[120.3157894736842, 182.76315789473685, 38.0]
pts[p]
[240.77142857142857, 261.9428571428571, 35.0]
pts[p]
[171.61764705882354, 259.1470588235294, 34.0]
pts[p]
[262.98387096774195, 234.33870967741936, 62.0]
pts[p]
[86.51785714285714, 282.48214285714283, 56.0]
pts[p]
[240.0, 264.9512195121951, 41.0]
pts[p]
[134.9189189189189, 182.51351351351352, 37.0]
pts[p]
[174.9, 260.6, 30.0]
[[174.9, 260.6, 30.0], [134.9189189189189, 182.51351351351352, 37.0], [86.51785714285714, 282.48214285714283, 56.0], [240.0, 264.9512195121951, 41.0], [262.98387096774195, 234.33870967741936, 62.0]]


pts[p]
[259.40625, 237.1875, 64.0]
pts[p]
[85.69811320754717, 282.45283018867923, 53.0]
pts[p]
[236.9268292682927, 265.390243902439, 41.0]
pts[p]
[170.6153846153846, 262.2307692307692, 39.0]
pts[p]
[140.25, 183.45, 20.0]
[[170.6153846153846, 262.2307692307692, 39.0], [140.25, 183.45, 20.0], [85.698113207

pts[p]
[72.88888888888889, 327.22222222222223, 54.0]
pts[p]
[232.30188679245282, 266.37735849056605, 53.0]
pts[p]
[59.61538461538461, 207.0, 39.0]
[[154.45454545454547, 254.6818181818182, 66.0], [59.61538461538461, 207.0, 39.0], [72.88888888888889, 327.22222222222223, 54.0], [221.06896551724137, 211.0344827586207, 58.0], [232.30188679245282, 266.37735849056605, 53.0]]


pts[p]
[149.66197183098592, 263.8732394366197, 71.0]
pts[p]
[231.52631578947367, 272.63157894736844, 57.0]
pts[p]
[217.36363636363637, 215.01818181818183, 55.0]
pts[p]
[67.16666666666667, 336.94444444444446, 54.0]
pts[p]
[55.5, 214.4, 30.0]
[[149.66197183098592, 263.8732394366197, 71.0], [55.5, 214.4, 30.0], [67.16666666666667, 336.94444444444446, 54.0], [217.36363636363637, 215.01818181818183, 55.0], [231.52631578947367, 272.63157894736844, 57.0]]


pts[p]
[147.15, 273.6, 80.0]
pts[p]
[231.0952380952381, 281.23809523809524, 63.0]
pts[p]
[216.0, 223.85454545454544, 55.0]
pts[p]
[60.339622641509436, 345.8490566037736, 53

[[], [181.8125, 240.25, 48.0], [99.83823529411765, 369.1764705882353, 68.0], [180.3939393939394, 331.72727272727275, 99.0], [279.1521739130435, 369.10869565217394, 138.0]]
[[34.8, 324.3, 10.0], [181.8125, 240.25, 48.0], [99.83823529411765, 369.1764705882353, 68.0], [180.3939393939394, 331.72727272727275, 99.0], [279.1521739130435, 369.10869565217394, 138.0]]


pts[p]
[290.6742857142857, 348.80571428571426, 175.0]
pts[p]
[197.2875, 324.9, 80.0]
pts[p]
[107.77611940298507, 352.8805970149254, 67.0]
pts[p]
[187.67307692307693, 237.80769230769232, 52.0]
hand2
hand1
foot1
foot2
converting
[[], [187.67307692307693, 237.80769230769232, 52.0], [107.77611940298507, 352.8805970149254, 67.0], [197.2875, 324.9, 80.0], [290.6742857142857, 348.80571428571426, 175.0]]
[[34.8, 324.3, 10.0], [187.67307692307693, 237.80769230769232, 52.0], [107.77611940298507, 352.8805970149254, 67.0], [197.2875, 324.9, 80.0], [290.6742857142857, 348.80571428571426, 175.0]]


pts[p]
[287.01895734597156, 321.1421800947867

pts[p]
[266.12068965517244, 218.3793103448276, 58.0]
pts[p]
[160.28571428571428, 288.48979591836735, 49.0]
pts[p]
[255.9375, 258.0, 48.0]
pts[p]
[165.34883720930233, 141.48837209302326, 43.0]
[[185.1864406779661, 223.0677966101695, 59.0], [165.34883720930233, 141.48837209302326, 43.0], [160.28571428571428, 288.48979591836735, 49.0], [255.9375, 258.0, 48.0], [266.12068965517244, 218.3793103448276, 58.0]]


pts[p]
[267.09090909090907, 217.13636363636363, 66.0]
pts[p]
[185.625, 224.46428571428572, 56.0]
pts[p]
[144.24, 293.4, 50.0]
pts[p]
[257.79545454545456, 257.04545454545456, 44.0]
pts[p]
[136.725, 144.375, 40.0]
[[185.625, 224.46428571428572, 56.0], [136.725, 144.375, 40.0], [144.24, 293.4, 50.0], [257.79545454545456, 257.04545454545456, 44.0], [267.09090909090907, 217.13636363636363, 66.0]]


pts[p]
[265.16129032258067, 212.90322580645162, 62.0]
pts[p]
[184.6315789473684, 224.8421052631579, 57.0]
pts[p]
[260.6170212765957, 251.68085106382978, 47.0]
pts[p]
[132.32608695652175, 296.021

[193.71428571428572, 144.71428571428572, 21.0]
hand1
hand2
torso
foot1
foot2
converting
[[193.71428571428572, 144.71428571428572, 21.0], [120.46428571428571, 76.71428571428571, 84.0], [126.57692307692308, 201.23076923076923, 52.0], [254.22680412371133, 161.2577319587629, 97.0], [254.22680412371133, 161.2577319587629, 97.0]]
[[193.71428571428572, 144.71428571428572, 21.0], [120.46428571428571, 76.71428571428571, 84.0], [126.57692307692308, 201.23076923076923, 52.0], [254.22680412371133, 161.2577319587629, 97.0], [254.22680412371133, 161.2577319587629, 97.0]]


pts[p]
[252.33333333333334, 157.52777777777777, 108.0]
pts[p]
[104.71621621621621, 98.51351351351352, 74.0]
pts[p]
[186.1764705882353, 144.76470588235293, 51.0]
pts[p]
[102.93333333333334, 192.4, 45.0]
hand2
hand1
torso
foot1
foot2
converting
[[186.1764705882353, 144.76470588235293, 51.0], [104.71621621621621, 98.51351351351352, 74.0], [102.93333333333334, 192.4, 45.0], [252.33333333333334, 157.52777777777777, 108.0], [252.3333333

pts[p]
[66.64285714285714, 82.07142857142857, 14.0]
[[134.1492537313433, 131.46268656716418, 67.0], [66.64285714285714, 82.07142857142857, 14.0], [103.078125, 203.8125, 64.0], [173.08064516129033, 77.75806451612904, 62.0], [199.42105263157896, 171.07894736842104, 38.0]]


pts[p]
[155.55, 125.925, 80.0]
pts[p]
[184.91666666666666, 60.833333333333336, 72.0]
pts[p]
[115.575, 209.7, 40.0]
pts[p]
[101.28, 64.92, 25.0]
pts[p]
[213.72, 151.32, 25.0]
[[155.55, 125.925, 80.0], [101.28, 64.92, 25.0], [115.575, 209.7, 40.0], [184.91666666666666, 60.833333333333336, 72.0], [213.72, 151.32, 25.0]]


pts[p]
[206.6341463414634, 50.19512195121951, 82.0]
pts[p]
[180.2608695652174, 125.95652173913044, 69.0]
pts[p]
[138.21428571428572, 57.57142857142857, 42.0]
pts[p]
[116.92105263157895, 208.73684210526315, 38.0]
pts[p]
[233.21052631578948, 125.52631578947368, 19.0]
[[180.2608695652174, 125.95652173913044, 69.0], [138.21428571428572, 57.57142857142857, 42.0], [116.92105263157895, 208.73684210526315, 38.0